In [1]:
# CELL 1: VERIFY LOCAL PDW FILE
import pandas as pd
from pathlib import Path

FILE_PATH = Path("PDW_data.parquet")

if not FILE_PATH.is_file():
    raise FileNotFoundError(f"File not found: {FILE_PATH}")

print(f"File Found: {FILE_PATH}")
df=pd.read_parquet(FILE_PATH)
print(df.head())

File Found: PDW_data.parquet
  Emitter_ID    TOA_us  Freq_MHz  PW_us  Amplitude  Power_dBm  AC_Lat  \
0    Radar_3   262.079   9051.28   5.03   0.004693     -16.57    12.0   
1    Radar_0   831.823   9100.10   5.02   0.001586     -26.00    12.0   
2    Radar_3   862.444   9050.63   5.03   0.004732     -16.50    12.0   
3    Radar_2  1118.014   9000.72   7.99   0.001480     -26.60    12.0   
4    Radar_1  1181.078   9050.51   5.04   0.001469     -26.66    12.0   

      AC_Lon     DOA_deg    Range_m  
0  76.500000  354.555952   55937.57  
1  76.500001    3.809488  167350.52  
2  76.500003  355.359951   55937.26  
3  76.500002    5.328868  178892.74  
4  76.500003    5.352844  178892.71  


In [12]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

BATCH_SIZE = 10000
STORE_PATH = "pdw_feature_store.parquet"

REQ_COLS = ["TOA_us", "PW_us", "Freq_MHz", "Power_dBm", "DOA_deg", "AC_Lat", "AC_Lon"]

print("Batch processing setup ready.")


def clean_batch(df):
    # Convert only required columns at once
    df[REQ_COLS] = df[REQ_COLS].apply(pd.to_numeric, errors="coerce")

    # Build ONE mask
    mask = (
        (df["TOA_us"].values >= 0) &
        (df["PW_us"].values > 0) &
        (df["Freq_MHz"].values > 0)
    )

    df = df.loc[mask]

    # Drop rows with NaNs only in required columns
    df = df.dropna(subset=REQ_COLS)

    # Sort by TOA (important for TOA_gap)
    df = df.sort_values("TOA_us", kind="mergesort")
    df.reset_index(drop=True, inplace=True)

    return df


def engineer_features(df):

    # Work with numpy arrays
    doa = df["DOA_deg"].values
    toa = df["TOA_us"].values
    pw = df["PW_us"].values
    power = df["Power_dBm"].values

    # -----------------------------------------------------
    # AOA Features (same as before)
    # -----------------------------------------------------
    rad = np.radians(doa)
    df["AOA_sin_w"] = np.sin(rad)
    df["AOA_cos_w"] = np.cos(rad)

    # -----------------------------------------------------
    # dTOA and log_dTOA  (same as before)
    # -----------------------------------------------------
    dtoa = np.empty_like(toa)
    dtoa[0] = 1e-6
    dtoa[1:] = np.diff(toa)
    np.clip(dtoa, 1e-6, None, out=dtoa)

    df["log_dTOA"] = np.log1p(dtoa)

    # -----------------------------------------------------
    # NEW FEATURE: TOA_gap (raw dTOA, not logged)
    # -----------------------------------------------------
    # This preserves PRI information for clustering.
    df["TOA_gap"] = dtoa.astype(np.float32)

    # -----------------------------------------------------
    # PW, Power logs
    # -----------------------------------------------------
    df["log_PW"] = np.log1p(pw)

    min_power = power.min()
    df["log_Power"] = np.log1p(power - min_power + 1)

    return df


Batch processing setup ready.


In [13]:
# CELL 2B: EXECUTE BATCH PROCESSING

print("Starting batch processing from Parquet...")

running_sum = None
running_sq = None
running_count = 0
writer = None

# -----------------------------
# UPDATED FEATURE LIST
# -----------------------------
FEATURE_COLS = [
    "Freq_MHz",
    "AOA_sin_w",
    "AOA_cos_w",
    "log_PW",
    "log_Power",
    "log_dTOA",
    "TOA_gap"      # <<— NEW TEMPORAL FEATURE
]

READ_COLS = REQ_COLS

parquet_file = pq.ParquetFile(FILE_PATH)

for i in range(parquet_file.num_row_groups):

    print(f"Processing row group {i+1}/{parquet_file.num_row_groups}")

    table = parquet_file.read_row_group(i, columns=READ_COLS)
    df_batch = table.to_pandas()

    df_batch = clean_batch(df_batch)
    if df_batch.empty:
        continue

    df_batch = engineer_features(df_batch)

    # Convert features to Matrix
    X = df_batch[FEATURE_COLS].to_numpy(dtype=np.float32, copy=False)

    # Running mean and std
    batch_sum = X.sum(axis=0)
    X_sq = X * X
    batch_sq = X_sq.sum(axis=0)

    if running_sum is None:
        running_sum = batch_sum
        running_sq = batch_sq
        running_count = X.shape[0]
    else:
        running_sum += batch_sum
        running_sq += batch_sq
        running_count += X.shape[0]

    # Convert back to Parquet format
    out_table = pa.Table.from_pandas(df_batch, preserve_index=False)

    # Build new parquet file
    if writer is None:
        writer = pq.ParquetWriter(STORE_PATH, out_table.schema)

    writer.write_table(out_table)

if writer:
    writer.close()

print("\nCELL 2 COMPLETE: Features Stored.")
print("Feature store saved to:", STORE_PATH)
print("Total pulses processed:", running_count)

Starting batch processing from Parquet...
Processing row group 1/2
Processing row group 2/2

CELL 2 COMPLETE: Features Stored.
Feature store saved to: pdw_feature_store.parquet
Total pulses processed: 1895990


In [14]:
# CELL 3: COMPUTE GLOBAL MEAN + STD FOR NORMALIZATION

import numpy as np
import pickle

print("Computing global statistics for scaling...")

if running_count == 0:
    raise RuntimeError("No pulses were processed.")

# float64 -> Numerical Accuracy
running_sum = running_sum.astype(np.float64, copy=False)
running_sq = running_sq.astype(np.float64, copy=False)

# Compute Mean
global_mean = running_sum / running_count

# Compute Variance safely
global_var = (running_sq / running_count) - (global_mean * global_mean)

# Prevent negative Variance
np.maximum(global_var, 0.0, out=global_var)

# Compute STD with numerical safety
global_std = np.sqrt(global_var + 1e-9)

norm_params = {
    "feature_cols": FEATURE_COLS,
    "mean": global_mean.astype(np.float32),
    "std": global_std.astype(np.float32),
    "count": running_count
}

print("\nGLOBAL NORMALIZATION PARAMETERS:")
print("Mean:", norm_params["mean"])
print("Std :", norm_params["std"])
print("Total pulses counted:", running_count)

with open("norm_params.pkl", "wb") as f:
    pickle.dump(norm_params, f, protocol=pickle.HIGHEST_PROTOCOL)

print("\nCELL 3 COMPLETE.")
print("Normalization parameters saved to: norm_params.pkl")


Computing global statistics for scaling...

GLOBAL NORMALIZATION PARAMETERS:
Mean: [9.0589863e+03 6.8318896e-02 7.2207624e-01 1.8645937e+00 1.8478618e+00
 4.5631485e+00 1.4377835e+02]
Std : [ 35.62655      0.21612506   0.6536271    0.15591793   0.59555817
   1.0876029  107.682785  ]
Total pulses counted: 1895990

CELL 3 COMPLETE.
Normalization parameters saved to: norm_params.pkl


In [15]:
# CELL 4: HDBSCAN RF CLUSTERING (Corrected for TOA-gap Feature)

import pandas as pd
import numpy as np
import hdbscan
import pickle
import gc
from sklearn.decomposition import PCA

# ------------------------------------------
# LOAD FEATURE STORE
# ------------------------------------------

print("Loading feature store from disk...")
df = pd.read_parquet(STORE_PATH)
print("Feature Store Loaded. Total pulses:", len(df))

# ------------------------------------------
# LOAD NORMALIZATION PARAMETERS
# ------------------------------------------

with open("norm_params.pkl", "rb") as f:
    norm_params = pickle.load(f)

FEATURE_COLS = norm_params["feature_cols"] + ["TOA_gap"]   # <-- ADD TEMPORAL FEATURE
global_mean = norm_params["mean"]
global_std  = norm_params["std"]

# Recompute mean/std for TOA_gap dynamically
gap_mean = df["TOA_gap"].mean()
gap_std  = df["TOA_gap"].std() + 1e-6    # avoid divide by zero

# ------------------------------------------
# NORMALIZE FEATURES
# ------------------------------------------

print("\nNormalizing features...")

X = df[FEATURE_COLS].to_numpy(dtype=np.float32)

# Use global mean/std for original features
# and dynamic mean/std for TOA_gap
X_norm = X.copy()
X_norm[:, :-1] = (X[:, :-1] - global_mean) / global_std   # original 6 features
X_norm[:, -1]  = (X[:, -1]  - gap_mean)   / gap_std       # TOA_gap normalization

print("Normalization complete.")

# ------------------------------------------
# PCA FOR DIMENSION REDUCTION (optional but highly beneficial)
# ------------------------------------------

print("\nApplying PCA (6+1 → 4 dims)...")

pca = PCA(n_components=4, random_state=42)
X_pca = pca.fit_transform(X_norm)

print("PCA complete. Variance retained:", np.sum(pca.explained_variance_ratio_))

# ------------------------------------------
# HDBSCAN CLUSTERING (Coarse stage)
# ------------------------------------------

print("\nRunning HDBSCAN clustering...")

n_samples = len(df)

min_cluster_size = max(500, int(0.001 * n_samples))
min_samples = max(10, int(np.sqrt(min_cluster_size)))

print("min_cluster_size:", min_cluster_size)
print("min_samples:", min_samples)

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=min_cluster_size,
    min_samples=min_samples,
    cluster_selection_epsilon=0.5,
    metric="euclidean",
    cluster_selection_method="leaf",
    approx_min_span_tree=True,
    gen_min_span_tree=False,
    core_dist_n_jobs=-1
)

labels = clusterer.fit_predict(X_pca)

print("HDBSCAN done.")
print("Unique coarse clusters found:", sorted(set(labels)))
print("Noise pulses:", np.sum(labels == -1))

# ------------------------------------------
# SAVE COARSE CLUSTERS
# ------------------------------------------

df["Coarse_Cluster"] = labels

CLUSTERED_OUTPUT = "pdw_clustered.parquet"
df.to_parquet(CLUSTERED_OUTPUT, index=False)

print("\nCELL 4 COMPLETE:")
print("Clustered PDWs saved to:", CLUSTERED_OUTPUT)

# Memory cleanup
del X_norm, X_pca, clusterer
gc.collect()

Loading feature store from disk...
Feature Store Loaded. Total pulses: 1895990

Normalizing features...
Normalization complete.

Applying PCA (6+1 → 4 dims)...
PCA complete. Variance retained: 0.9128465

Running HDBSCAN clustering...
min_cluster_size: 1895
min_samples: 43
HDBSCAN done.
Unique coarse clusters found: [np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3)]
Noise pulses: 1

CELL 4 COMPLETE:
Clustered PDWs saved to: pdw_clustered.parquet


0

In [16]:
# CELL 4.5: PRINT FIRST 5 AND LAST 5 ROWS OF EACH CLUSTER

import pandas as pd
import numpy as np

print("\nInspecting clusters...\n")

if "Coarse_Cluster" not in df.columns:
    raise RuntimeError("Coarse_Cluster column not found. Run Cell 4 first.")

unique_clusters = sorted(df["Coarse_Cluster"].unique())

for cluster_id in unique_clusters:
    
    if cluster_id == -1:
        continue  # Skip noise (optional)

    cluster_df = df[df["Coarse_Cluster"] == cluster_id].copy()

    if cluster_df.empty:
        continue

    cluster_df.sort_values("TOA_us", inplace=True)

    print("=" * 70)
    print(f"CLUSTER {cluster_id}")
    print(f"Total Pulses: {len(cluster_df)}")
    print("-" * 70)

    print(cluster_df.head(-5))
    print("\n")



Inspecting clusters...

CLUSTER 0
Total Pulses: 340753
----------------------------------------------------------------------
               TOA_us  PW_us  Freq_MHz  Power_dBm    DOA_deg  AC_Lat  \
3        1.118014e+03   7.99   9000.72     -26.60   5.328868    12.0   
9        1.928859e+03   8.03   8998.73     -26.71   5.399738    12.0   
14       2.729081e+03   7.95   8998.33     -26.46   5.327662    12.0   
20       3.531189e+03   8.06   9000.40     -26.56   5.267159    12.0   
26       4.319028e+03   8.02   8999.70     -26.68   5.330958    12.0   
...               ...    ...       ...        ...        ...     ...   
1895938  2.725954e+08   7.91   8998.77     -14.08  22.889325    12.0   
1895944  2.725962e+08   8.02   9000.62     -14.07  22.983007    12.0   
1895950  2.725970e+08   7.97   9001.41     -13.95  22.948837    12.0   
1895955  2.725978e+08   7.94   8999.47     -14.13  23.082891    12.0   
1895960  2.725986e+08   7.95   9001.13     -14.06  22.674282    12.0   

        

In [27]:
# CELL 5: FAST PRI DE-INTERLEAVING (with correct stagger handling)
# ---------------------------------------------------------------

import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import gc

print("Loading clustered PDW file...")
df = pd.read_parquet("pdw_clustered.parquet")
df["Fine_Cluster"] = -1

summary_rows = []
emitter_id = 0

# --------------------------
# PRI peak detection
# --------------------------
def get_pri_candidates(toa, bin_us=1.0):
    dtoa = np.diff(toa)
    dtoa = dtoa[(dtoa > 2) & (dtoa < 8000)]
    if len(dtoa) < 30:
        return []

    bins = np.arange(0, dtoa.max() + bin_us, bin_us)
    hist, edges = np.histogram(dtoa, bins=bins)

    peaks, _ = find_peaks(hist, prominence=np.max(hist)*0.30)
    return edges[peaks]

# --------------------------
# Grouping close PRI values
# --------------------------
def group_pri_candidates(pri_raw, window=20):
    if len(pri_raw) == 0:
        return []

    pri_raw = np.sort(pri_raw)
    groups = []
    cur = [pri_raw[0]]

    for p in pri_raw[1:]:
        if abs(p - cur[-1]) <= window:
            cur.append(p)
        else:
            groups.append(cur)
            cur = [p]

    groups.append(cur)
    return [np.mean(g) for g in groups]

# --------------------------
# Scoring PRI strength
# --------------------------
def score_pris(dtoa, pri_list, tol=0.10):
    scores = []
    for pri in pri_list:
        lo, hi = pri*(1-tol), pri*(1+tol)
        count = np.sum((dtoa >= lo) & (dtoa <= hi))
        scores.append((pri, count))

    max_count = max(s for _, s in scores)
    return [(p, s) for p, s in scores if s > max_count*0.25]


# --------------------------
# MAIN LOOP
# --------------------------
coarse_vals = sorted(df["Coarse_Cluster"].unique())
fine_cluster_id = 0

for c in coarse_vals:
    if c == -1:
        continue

    print("\n==============================")
    print(f"PROCESSING COARSE CLUSTER {c}")
    print("==============================")

    sub = df[df["Coarse_Cluster"] == c].copy()
    toa = np.sort(sub["TOA_us"].to_numpy())

    if len(toa) < 20:
        pri_type = "Unknown"
        df.loc[sub.index, "Fine_Cluster"] = fine_cluster_id

        summary_rows.append([
            emitter_id, c, len(sub),
            pri_type, None, []
        ])
        emitter_id += 1
        fine_cluster_id += 1
        continue

    # Step 1
    pri_raw = get_pri_candidates(toa)

    # Step 2
    pri_groups = group_pri_candidates(pri_raw)

    # Step 3
    dtoa = np.diff(toa)
    scored = score_pris(dtoa, pri_groups)

    if len(scored) == 0:
        df.loc[sub.index, "Fine_Cluster"] = fine_cluster_id
        summary_rows.append([
            emitter_id, c, len(sub), "Unknown", None, []
        ])
        emitter_id += 1
        fine_cluster_id += 1
        continue

    # Extract PRI list and frame PRI
    final_pris = sorted([p for p, _ in scored])
    frame_pri = np.mean(final_pris)

    # PRI type classification
    if len(final_pris) == 1:
        pri_type = "Fixed"
    else:
        pri_type = "Staggered"

    print(f"Detected PRI Type: {pri_type}")
    print(f"Frame PRI ≈ {frame_pri:.3f}")
    print(f"Sub PRIs: {final_pris}")

    # Assign ALL pulses to ONE fine cluster for staggered / fixed
    # ------------------------------------------------------------
    mask = np.zeros(len(dtoa), dtype=bool)

    for pri in final_pris:
        lo, hi = pri*0.85, pri*1.15
        mask |= (dtoa >= lo) & (dtoa <= hi)

    assigned_idx = sub.index[np.where(mask)[0]]
    df.loc[assigned_idx, "Fine_Cluster"] = fine_cluster_id

    summary_rows.append([
        emitter_id,
        c,
        len(assigned_idx),
        pri_type,
        float(frame_pri),
        final_pris
    ])

    emitter_id += 1
    fine_cluster_id += 1


# --------------------------
# SAVE
# --------------------------
summary_df = pd.DataFrame(summary_rows, columns=[
    "Emitter_ID", "Cluster", "Pulse_Count",
    "PRI_Type", "Frame_PRI", "Sub_PRIs"
])

print("\nFINAL EMITTER COUNT:", len(summary_df))
print("\nEmitter Summary:")
print(summary_df)

OUTPUT = "pdw_final_clusters.parquet"
df.to_parquet(OUTPUT, index=False)

Loading clustered PDW file...

PROCESSING COARSE CLUSTER 0
Detected PRI Type: Fixed
Frame PRI ≈ 800.000
Sub PRIs: [np.float64(800.0)]

PROCESSING COARSE CLUSTER 1
Detected PRI Type: Staggered
Frame PRI ≈ 649.000
Sub PRIs: [np.float64(599.0), np.float64(649.0), np.float64(699.0)]

PROCESSING COARSE CLUSTER 2
Detected PRI Type: Fixed
Frame PRI ≈ 600.000
Sub PRIs: [np.float64(600.0)]

PROCESSING COARSE CLUSTER 3
Detected PRI Type: Fixed
Frame PRI ≈ 399.000
Sub PRIs: [np.float64(399.0)]

FINAL EMITTER COUNT: 4

Emitter Summary:
   Emitter_ID  Cluster  Pulse_Count   PRI_Type  Frame_PRI  \
0           0        0       340752      Fixed      800.0   
1           1        1       419395  Staggered      649.0   
2           2        2       454329      Fixed      600.0   
3           3        3       681509      Fixed      399.0   

                Sub_PRIs  
0                [800.0]  
1  [599.0, 649.0, 699.0]  
2                [600.0]  
3                [399.0]  


In [28]:
# CELL 6: FINAL RADAR SUMMARY REPORT (Optimized + Tabular)

import numpy as np
import pandas as pd

print("\n==============================")
print("FINAL RADAR PARAMETER SUMMARY")
print("==============================\n")

# summary_df was produced in Cell 5 as the Emitter Summary
emitter_summary = summary_df.copy()

# Precompute cluster-level average parameters
cluster_stats = (
    df.groupby("Coarse_Cluster")[["Freq_MHz", "PW_us", "Power_dBm"]]
    .mean()
)

final_rows = []

for row in emitter_summary.itertuples(index=False):

    emitter_id   = row.Emitter_ID
    cluster_id   = row.Cluster
    pri_type     = row.PRI_Type
    frame_pri    = row.Frame_PRI
    sub_pris     = row.Sub_PRIs

    # Pull RF/PW/Power statistics for this cluster
    if cluster_id in cluster_stats.index:
        stats = cluster_stats.loc[cluster_id]
        rf_mhz   = round(stats["Freq_MHz"], 3)
        pw_us    = round(stats["PW_us"], 3)
        power_db = round(stats["Power_dBm"], 3)
    else:
        # Fallback (should not occur)
        rf_mhz, pw_us, power_db = None, None, None

    # Ensure PRI list formatting
    if isinstance(sub_pris, (list, np.ndarray)):
        pri_values = np.round(sub_pris, 3).tolist()
    else:
        pri_values = []

    # Handle missing frame PRI
    frame_pri_val = round(frame_pri, 3) if frame_pri is not None else None

    final_rows.append({
        "Emitter_ID": emitter_id,
        "Coarse_Cluster": cluster_id,
        "RF_MHz": rf_mhz,
        "PW_us": pw_us,
        "Power_dBm": power_db,
        "PRI_Type": pri_type,
        "Frame_PRI_us": frame_pri_val,
        "PRI_Values_us": pri_values,
        "Pulse_Count": row.Pulse_Count
    })

final_df = pd.DataFrame(final_rows)

# Display formatted table
print(final_df.to_string(index=False))

print("\n===================================")
print("TOTAL RADARS DETECTED:", len(final_df))
print("===================================")


FINAL RADAR PARAMETER SUMMARY

 Emitter_ID  Coarse_Cluster   RF_MHz  PW_us  Power_dBm  PRI_Type  Frame_PRI_us         PRI_Values_us  Pulse_Count
          0               0 9000.000    8.0    -21.719     Fixed         800.0               [800.0]       340752
          1               1 9050.001    5.0    -10.976 Staggered         649.0 [599.0, 649.0, 699.0]       419395
          2               2 9049.997    5.0    -21.768     Fixed         600.0               [600.0]       454329
          3               3 9100.002    5.0    -20.615     Fixed         399.0               [399.0]       681509

TOTAL RADARS DETECTED: 4


In [29]:
# CELL 7: AOA-ONLY LOCALIZATION (WITH HEADING ESTIMATION)

import numpy as np
import pandas as pd

print("\n==============================")
print("AOA GEO-LOCALIZATION (BODY DOA FIXED)")
print("==============================")

df = pd.read_parquet("pdw_clustered.parquet")

# WGS84 constants
a = 6378137.0
e2 = 6.69437999014e-3


# ------------------------------------------------------------
# 1️⃣ Compute Aircraft Heading from Motion
# ------------------------------------------------------------
def compute_heading(lat, lon):

    lat = np.radians(lat)
    lon = np.radians(lon)

    dlon = np.diff(lon)
    x = np.sin(dlon) * np.cos(lat[1:])
    y = np.cos(lat[:-1]) * np.sin(lat[1:]) - \
        np.sin(lat[:-1]) * np.cos(lat[1:]) * np.cos(dlon)

    heading = np.degrees(np.arctan2(x, y))
    heading = (heading + 360) % 360

    # Pad first element
    heading = np.insert(heading, 0, heading[0])

    return heading


# ------------------------------------------------------------
# 2️⃣ Geodetic -> ECEF
# ------------------------------------------------------------
def geodetic_to_ecef(lat, lon, alt=0):

    lat = np.radians(lat)
    lon = np.radians(lon)

    N = a / np.sqrt(1 - e2 * np.sin(lat)**2)

    X = (N + alt) * np.cos(lat) * np.cos(lon)
    Y = (N + alt) * np.cos(lat) * np.sin(lon)
    Z = (N * (1 - e2) + alt) * np.sin(lat)

    return np.stack([X, Y, Z], axis=-1)


# ------------------------------------------------------------
# 3️⃣ ENU -> ECEF Conversion
# ------------------------------------------------------------
def enu_to_ecef(u_enu, lat, lon):

    lat = np.radians(lat)
    lon = np.radians(lon)

    sin_lat = np.sin(lat)
    cos_lat = np.cos(lat)
    sin_lon = np.sin(lon)
    cos_lon = np.cos(lon)

    t = np.array([
        [-sin_lon, -sin_lat*cos_lon,  cos_lat*cos_lon],
        [ cos_lon, -sin_lat*sin_lon,  cos_lat*sin_lon],
        [ 0,        cos_lat,          sin_lat]
    ])

    return (t @ u_enu.T).T


# ------------------------------------------------------------
# 4️⃣ Triangulation Solver
# ------------------------------------------------------------
def triangulate(P, U):

    A = np.zeros((3,3))
    b = np.zeros(3)
    I = np.eye(3)

    for i in range(len(P)):
        u = U[i].reshape(3,1)
        Pi = P[i]

        Ai = I - u @ u.T
        A += Ai
        b += Ai @ Pi

    try:
        X = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        return None

    return X


# ------------------------------------------------------------
# 5️⃣ ECEF -> Geodetic
# ------------------------------------------------------------
def ecef_to_geodetic(X, Y, Z):

    lon = np.arctan2(Y, X)
    p = np.sqrt(X**2 + Y**2)
    lat = np.arctan2(Z, p * (1 - e2))

    for _ in range(5):
        N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
        lat = np.arctan2(Z + e2 * N * np.sin(lat), p)

    return np.degrees(lat), np.degrees(lon)


# ------------------------------------------------------------
# 6️⃣ LOCALIZE EACH CLUSTER
# ------------------------------------------------------------

localized_emitters = []

for cid in df["Coarse_Cluster"].unique():

    if cid == -1:
        continue

    cluster = df[df["Coarse_Cluster"] == cid].sort_values("TOA_us")

    if len(cluster) < 20:
        continue

    lat = cluster["AC_Lat"].values
    lon = cluster["AC_Lon"].values
    doa_body = cluster["DOA_deg"].values

    # Compute aircraft heading
    heading = compute_heading(lat, lon)

    # Convert body DOA -> global azimuth
    azimuth = (heading + doa_body) % 360

    # Build ENU unit vectors
    az_rad = np.radians(azimuth)

    u_enu = np.stack([
        np.sin(az_rad),   # East
        np.cos(az_rad),   # North
        np.zeros_like(az_rad)
    ], axis=-1)

    # Convert aircraft positions to ECEF
    P = geodetic_to_ecef(lat, lon)

    # Convert ENU vectors to ECEF
    U = np.array([
        enu_to_ecef(u_enu[i], lat[i], lon[i])
        for i in range(len(u_enu))
    ])

    # Solve intersection
    X = triangulate(P, U)

    if X is None:
        continue

    est_lat, est_lon = ecef_to_geodetic(X[0], X[1], X[2])

    localized_emitters.append({
        "Cluster_ID": cid,
        "Latitude": round(est_lat, 6),
        "Longitude": round(est_lon, 6),
        "Pulse_Count": len(cluster)
    })


radar_locations = pd.DataFrame(localized_emitters)

print("\n==============================")
print("LOCALIZATION COMPLETE")
print("==============================")
print(radar_locations)
print("\nTOTAL RADARS LOCALIZED:", len(radar_locations))



AOA GEO-LOCALIZATION (BODY DOA FIXED)

LOCALIZATION COMPLETE
   Cluster_ID   Latitude  Longitude  Pulse_Count
0           1  12.048834  76.998925       419396
1           0  11.848987  78.107243       340753
2           2  11.848988  78.107237       454330
3           3  11.899021  78.006173       681510

TOTAL RADARS LOCALIZED: 4
